In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
from datetime import datetime, timedelta

dart = pd.DataFrame()
dart_temp = pd.DataFrame()
dart_temp = dart_temp.append({"Cat":"","comName":"","coID":"","Content":"","rcpNo":"","pubTime":"","pubDate":"","reqDate":""},ignore_index=True)
print("--------------------------------------------------------")
print("일정 기간 동안의 dart 데이터 불러와서 합치기")
print("시작 년,월,일과 종료 일을 입력하세요 [같은 달만 됩니다.]")
print("--------------------------------------------------------")
date1=input("년을 입력하세요(4자리 모두 입력하세요)")
date2=input("월을 입력하세요")
date3=input("시작일을 입력하세요")
date4=input("종료일을 입력하세요")
date2=date2.zfill(2)
date3=date3.zfill(2)
date4=date4.zfill(2)
date_start=datetime(int(date1),int(date2),int(date3))
date_end=datetime(int(date1),int(date2),int(date4))
day1=date_end-date_start

for a in range(0,day1.days+1):
    date=date1+date2+str(int(date3)+a).zfill(2)
    url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
    url_part2="""&sort=&series=&mdayCnt=0&currentPage="""
    url=url_part1+date+url_part2
    res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
    soup=bs(res.text,'html.parser')
    if(soup.find_all("p")[8].text.strip()=="상기 보고서명 앞의 대괄호([]) 및 비고란의 약어(예:)에 마우스를 위치하면 설명이 나타나니 참고하시기 바랍니다."):
        continue
    else:
        totNum=soup.find_all("p")[8].text
    totNum=re.sub(r"\[.*\] \[총", "",totNum)  # '['로 시작하고 ']'' 끝나고 '[총'로 시작하는 것 찾아서 삭제
    totNum=re.sub(r"건]","",totNum)
    totNum=totNum.strip()
    totNum=int(totNum.strip())
    totPage=(totNum//100)+1
    num=len(soup.find_all("tr"))
    
    for k in range(1,totPage+1):
        url_part1="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
        url_part2="""&sort=&series=&mdayCnt=0&currentPage="""+str(k)
        url=url_part1+date+url_part2
        res=requests.get(url) #urllib.request.urlopen() 과 동일한 역할
        soup=bs(res.text,'html.parser')
        num=len(soup.find_all("tr"))
        
        for i in range(1,num):
            if len(soup.find_all("tr")[i].find_all("td"))>=2:
                dart_temp.at[0,"Cat"]=soup.find_all("tr")[i].find_all("td")[1].find("img").get("title")   
                dart_temp.at[0,"comName"]=soup.find_all("tr")[i].find("a").text.strip()
                dart_temp.at[0,"coID"]=re.findall(r"[0-9]{8}",soup.find_all("tr")[i].find("a").get("href"))[0]
                dart_temp.at[0,"Content"]=re.sub("\\t","",re.sub("\\n","",re.sub("\\r","",soup.find_all("tr")[i].find_all("td")[2].text.replace(" ",""))))
                dart_temp.at[0,"rcpNo"]=re.findall(r"[0-9]{14}",soup.find_all("tr")[i].find_all("td")[2].find("a").get("href"))[0]
                dart_temp.at[0,"pubTime"]=soup.find_all("tr")[i].find("td").text.strip()
                dart_temp.at[0,"pubDate"]=date
                dart_temp.at[0,"reqDate"]=soup.find_all("tr")[i].find_all("td",class_="cen_txt")[1].text
            dart=dart.append(dart_temp, ignore_index=True)

            for j in range(0,len(dart_temp.columns)):
                dart_temp.iat[0,j]=""
dart

--------------------------------------------------------
일정 기간 동안의 dart 데이터 불러와서 합치기
시작 년,월,일과 종료 일을 입력하세요 [같은 달만 됩니다.]
--------------------------------------------------------
년을 입력하세요2017
월을 입력하세요03
시작일을 입력하세요1
종료일을 입력하세요02


,Cat,Content,coID,comName,pubDate,pubTime,rcpNo,reqDate
0,코스닥시장,타법인주식및출자증권취득결정(자율공시),00226228,한프,20170302,20:46,20170302901111,2017.03.02
1,코스닥시장,주권관련사채권의취득결정,00226228,한프,20170302,20:45,20170302901133,2017.03.02
2,코스닥시장,기타시장안내(상장적격성실질심사관련),00188715,현진소재,20170302,19:01,20170302901109,2017.03.02
3,코스닥시장,주권매매거래정지(투자자보호),00188715,현진소재,20170302,19:01,20170302901128,2017.03.02
4,코스닥시장,최대주주변경을수반하는주식양수도계약체결,00188715,현진소재,20170302,18:55,20170302901127,2017.03.02
...,...,...,...,...,...,...,...,...
832,코스닥시장,[기재정정]사외이사의선임ㆍ해임또는중도퇴임에관한신고,00608316,대한과학,20170302,07:00,20170228001518,2017.03.02
833,기타법인,[기재정정]특수관계인으로부터자금차입,00868237,부영환경산업,20170302,07:00,20170228001522,2017.03.02
834,기타법인,[기재정정]특수관계인으로부터자금차입,00868237,부영환경산업,20170302,07:00,20170228001520,2017.03.02
835,기타법인,대규모기업집단현황공시[분기별공시(개별회사용)],01168833,신성장동력그린퓨처사모투자전문회사,20170302,07:00,20170228001524,2017.03.02
